In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import os
import pandas as pd
import numpy as np
import re
import time
import pickle

from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets, svm
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_selection import SelectPercentile, SelectFpr, chi2, mutual_info_classif
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, BaggingClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from tqdm import tqdm,trange
from collections import defaultdict,Counter
from text_to_num import text2num,alpha2digit
from mlxtend.classifier import StackingCVClassifier

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

In [2]:
def preprocess(file, 
               process = None, 
               convert_to_seconds = False):
    curr_dir = os.path.dirname(os.path.realpath('__file__'))
    if not process:
        train_file = os.path.join(curr_dir, 'COMP30027_2021_Project2_datasets\\'+file)
    else:
        train_file = os.path.join(curr_dir, 'COMP30027_2021_Project2_datasets\\recipe_text_features_' + process + '\\' + file)

    data = pd.read_csv(train_file, index_col = False, delimiter = ',')
    if convert_to_seconds:
        tqdm.pandas(desc="Converting...")
        data['seconds'] = data['steps'].progress_apply(convert_step_to_time)
    return data

In [3]:
time_retrieval = lambda words, sentence : re.findall(re.compile('|'.join(
        [n + w + "\\b" for w in words for n in ["[0-9]+\s*","another ","few ","a ","an ","several "]])), sentence)

kw_retrieval = lambda words, sentence : re.findall(re.compile('|'.join(
        ["\\b" + w + "\\b" for w in words])), sentence)

def convert_step_to_time(step, keyword_check = True):
    total_time = 0 # in seconds
    times = defaultdict(list)
    time_unit = {"second":1, "minute":60, "hour":60*60}
    kw_dict = {"overnight":8, "night":8, "nights":8, "freeze":5, "refrigerate":3,
               "day":12, "cook on low":7, "slow cook":7, "crockpot":7, "crock pot":7,
               "cook low":7, "boil":1, "heat":1, "bread machine":1.5
               }
    
    # Iterate through each step to find time value using RE
    numeric_step = alpha2digit(step,'en')
    times["second"] = time_retrieval(["more seconds","more second","more secs","more sec","seconds","second","secs","sec","s"],numeric_step)
    times["minute"] = time_retrieval(["more minutes","more minute","more min","more mins","minutes","minute","min","mins","m","ms"],numeric_step)
    times["hour"] = time_retrieval(["more hours","more hour","more hrs","more hr","hours","hour","hrs","hr","h","hs"],numeric_step)
    special_keywords = kw_retrieval(kw_dict.keys(),numeric_step) if keyword_check else None

    for unit in time_unit.keys():
        #total_time += sum([float(re.findall(r'[0-9]+',t.split()[0])[0]) * time_unit[unit] for t in times[unit]])
        count = 0
        for t in times[unit]:
            numerator = t.split()[0]

            if numerator in ["another","a","an"]:
                count = 1
            elif numerator in ["few","several"]:
                count = 4
            else:
                count = float(re.findall(r'[0-9]+',numerator)[0])

            total_time += count * time_unit[unit]

    if keyword_check and not total_time and special_keywords:
        total_time += 60*60*kw_dict[special_keywords[0]]

    if total_time == 0:
        return None
    return total_time

In [3]:
train_data = pd.read_csv('COMP30027_2021_Project2_datasets\\recipe_train.csv')
train_data_50 = pd.read_csv('COMP30027_2021_Project2_datasets\\recipe_text_features_doc2vec50\\train_steps_doc2vec50.csv',header=None, names = ['coord'+str(i) for i in range(50)])

In [10]:
train = pd.concat([train_data,train_data_50],axis=1)
y = train['duration_label']
x = train.drop(['name','steps','ingredients','duration_label'], axis=1)

,n_steps,n_ingredients,coord0,coord1,coord2,coord3,coord4,coord5,coord6,coord7,...,coord40,coord41,coord42,coord43,coord44,coord45,coord46,coord47,coord48,coord49
0,6,12,0.294882,0.291131,0.779709,0.634566,-0.658954,-2.018979,0.366450,0.125453,...,0.524914,-0.059248,-0.851974,-0.878965,0.110095,0.177768,-0.369561,0.355961,-0.481994,0.356241
1,9,5,0.752555,-1.072124,1.875412,0.247283,0.184581,1.093477,0.985011,-0.111624,...,-1.743889,0.805147,-0.825264,0.058763,1.868224,-1.211003,-2.073559,0.249009,-1.054287,-0.293384
2,15,10,0.002207,0.722088,0.810733,-0.191287,-1.167714,-1.248609,-1.362723,-1.567349,...,0.333650,-0.673702,-2.324360,0.687601,-0.939967,-1.561040,-0.732557,-0.939450,0.274603,-0.620148
3,10,8,-0.293604,0.393680,0.460519,0.049180,0.561528,-1.391731,-0.359667,-0.185326,...,0.607433,-0.018920,-2.685916,-0.463611,-0.059493,1.212257,-0.245923,-0.432690,-0.592135,0.654083
4,6,5,0.388302,0.030247,0.680402,-0.275467,0.248622,-0.108000,-0.078504,-0.457251,...,0.214621,-0.520235,0.130486,0.211613,0.499143,0.743107,-0.426484,0.528443,-0.694904,0.264700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,6,13,-0.585818,0.790814,0.655565,-1.305637,0.910338,-0.540507,0.469684,-0.645112,...,0.066317,-0.529131,-1.078020,0.445699,-0.757968,0.557478,0.097131,0.444583,0.556911,0.224863
39996,15,16,-1.499155,-0.543401,1.016098,0.893885,-0.412951,-0.444333,0.244239,-0.988383,...,-1.246466,1.540797,-1.392812,-0.494735,-0.114502,-0.351079,-0.845888,-0.123080,0.474510,0.019088
39997,5,8,0.551546,0.789452,0.534483,-0.565389,-1.286248,0.214225,1.214235,-0.286979,...,0.454007,-1.052013,0.496355,1.004171,0.237426,-1.293243,0.792720,-0.514634,1.201972,0.139259
39998,7,17,-0.258097,-0.503870,0.473143,0.121228,-1.064937,-1.245729,-0.570034,-0.956063,...,-0.036450,0.486010,-1.246498,-0.175695,-0.475982,0.737286,-0.400827,0.635068,-0.527436,0.370146


In [18]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=42)

In [5]:
start = time.time()
c1 = RandomForestClassifier(n_estimators=100,min_samples_split=50,min_samples_leaf=1, criterion='gini', n_jobs=-1, verbose=True)
c2 = GaussianNB()
c3 = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000, n_jobs=-1, verbose=False)
lr = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000, n_jobs=-1, verbose=False)

sclf = StackingCVClassifier(classifiers=[c1,c2,c3],
                            meta_classifier=lr, use_probas=True, verbose=False)

#sclf.fit(X_train,y_train)
print('Process takes ',time.time()-start, 'seconds')

Process takes  0.0 seconds


In [31]:
preds = pd.Series(sclf.predict(X_test))
classification_report(y_test, preds)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished


'              precision    recall  f1-score   support\n\n         1.0       0.72      0.71      0.71      2646\n         2.0       0.72      0.77      0.74      3053\n         3.0       0.49      0.20      0.28       301\n\n    accuracy                           0.71      6000\n   macro avg       0.64      0.56      0.58      6000\nweighted avg       0.71      0.71      0.71      6000\n'

In [32]:
print(f'Accuracy: {100*accuracy_score(preds, y_test):.2f}%')

Accuracy: 71.32%


In [33]:
cross_val_score(sclf,x,y).mean()

Fitting 3 classifiers...
Fitting classifier1: randomforestclassifier (1/3)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.1s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Par

0.71325

In [35]:
cross_val_score(AdaBoostClassifier(),x,y).mean()

0.6760249999999999

In [36]:
cross_val_score(BaggingClassifier(c2),x,y).mean()

0.63335

In [9]:
train_data = preprocess('recipe_train.csv', convert_to_seconds=True)#pd.read_csv('COMP30027_2021_Project2_datasets\\recipe_train.csv')
train_data_100 = pd.read_csv('COMP30027_2021_Project2_datasets\\recipe_text_features_doc2vec100\\train_steps_doc2vec100.csv',header=None, names = ['coord'+str(i) for i in range(100)])
train = pd.concat([train_data,train_data_100],axis=1)
y100 = train['duration_label']
x100 = train.drop(['name','steps','ingredients','duration_label'], axis=1)

Converting...: 100%|██████████| 40000/40000 [00:49<00:00, 815.33it/s]


In [10]:
train = train.dropna()
y100 = train['duration_label']
x100 = train.drop(['name','steps','ingredients','duration_label'], axis=1)

In [12]:
start = time.time()
print(cross_val_score(sclf,x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    4.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.3s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]:

In [13]:
start = time.time()
print(cross_val_score(SVC(),x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

0.7294058455715811

In [15]:
start = time.time()
print(cross_val_score(GaussianNB(),x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

0.5699892538439091
Process takes  0.5106613636016846 seconds


In [16]:
start = time.time()
print(cross_val_score(DecisionTreeClassifier(),x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

0.6779986540110515
Process takes  29.413482666015625 seconds


In [17]:
start = time.time()
print(cross_val_score(LogisticRegression(max_iter=10000),x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

C:\Users\HP OMEN 15\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
C:\Users\HP OMEN 15\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    ht

In [18]:
start = time.time()
print(cross_val_score(RandomForestClassifier(),x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

0.7441915533184241
Process takes  138.90527200698853 seconds


In [19]:
start = time.time()
print(cross_val_score(AdaBoostClassifier(),x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

0.7364848804798234
Process takes  91.97082996368408 seconds


In [21]:
start = time.time()
print(cross_val_score(KNeighborsClassifier(),x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

0.7552663885195186
Process takes  12.221656560897827 seconds


In [20]:
start = time.time()
print(cross_val_score(BaggingClassifier(DecisionTreeClassifier()),x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

0.7482447571595648
Process takes  179.22210359573364 seconds


In [22]:
start = time.time()
print(cross_val_score(BaggingClassifier(KNeighborsClassifier()),x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

0.7566651080659482
Process takes  114.75170159339905 seconds


In [23]:
start = time.time()
print(cross_val_score(BaggingClassifier(SVC()),x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

0.7290062410979176
Process takes  3350.5638234615326 seconds


In [34]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
start = time.time()
param_grid = {'n_neighbors': np.arange(38, 238)}
rcv = GridSearchCV(KNeighborsClassifier(),param_grid, cv=5)
rcv.fit(x100,y100)
print('Process takes ',time.time()-start, 'seconds')
print(rcv.best_params_)
rcv.best_score_

Process takes  3129.6548767089844 seconds
{'n_neighbors': 38}


0.7703660191201482

In [47]:
start = time.time()
c1 = RandomForestClassifier(n_estimators=100,min_samples_split=50,min_samples_leaf=1, criterion='gini', n_jobs=-1)
c2 = GaussianNB()
c3 = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000, n_jobs=-1)
lr = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000, n_jobs=-1)

sclf = StackingCVClassifier(classifiers=[c1,c2,c3],
                            meta_classifier=lr, use_probas=True)

start = time.time()
print(cross_val_score(sclf,x100,y100, cv = 10).mean())
print('Process takes ',time.time()-start, 'seconds')

0.7675407625166687
Process takes  425.40338587760925 seconds


In [37]:
start = time.time()
c1 = RandomForestClassifier(n_estimators=100,min_samples_split=50,min_samples_leaf=1, criterion='gini', n_jobs=-1)
c2 = AdaBoostClassifier()
c3 = KNeighborsClassifier(n_neighbors=38)
lr = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000, n_jobs=-1)

sclf = StackingCVClassifier(classifiers=[c1,c2,c3],
                            meta_classifier=lr, use_probas=True)

start = time.time()
print(cross_val_score(sclf,x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

0.7761890203919485
Process takes  291.49292397499084 seconds


In [43]:
start = time.time()
c1 = RandomForestClassifier(n_estimators=100,min_samples_split=50,min_samples_leaf=1, criterion='gini', n_jobs=-1)
c2 = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000, n_jobs=-1)
c3 = KNeighborsClassifier(n_neighbors=38)
lr = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000, n_jobs=-1)

sclf = StackingCVClassifier(classifiers=[c1,c2,c3],
                            meta_classifier=lr, use_probas=True)

start = time.time()
print(cross_val_score(sclf,x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

0.7817265968809544
Process takes  246.60619163513184 seconds


In [28]:
start = time.time()
c1 = RandomForestClassifier(n_estimators=100,min_samples_split=50,min_samples_leaf=1, criterion='gini', n_jobs=-1)
c2 = AdaBoostClassifier()
c3 = KNeighborsClassifier(n_neighbors=21)
lr = KNeighborsClassifier(n_neighbors=21)

sclf = StackingCVClassifier(classifiers=[c1,c2,c3],
                            meta_classifier=lr, use_probas=True)

start = time.time()
print(cross_val_score(sclf,x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

0.7719644451629282
Process takes  267.93580436706543 seconds


In [29]:
start = time.time()
c1 = RandomForestClassifier(n_estimators=100,min_samples_split=50,min_samples_leaf=1, criterion='gini', n_jobs=-1)
c2 = AdaBoostClassifier()
c3 = KNeighborsClassifier(n_neighbors=21)
lr = BaggingClassifier(DecisionTreeClassifier())

sclf = StackingCVClassifier(classifiers=[c1,c2,c3],
                            meta_classifier=lr, use_probas=True)

start = time.time()
print(cross_val_score(sclf,x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

0.7517270543982342
Process takes  276.2685396671295 seconds


In [30]:
start = time.time()
c1 = RandomForestClassifier(n_estimators=100,min_samples_split=50,min_samples_leaf=1, criterion='gini', n_jobs=-1)
c2 = AdaBoostClassifier()
lr = KNeighborsClassifier(n_neighbors=21)
c3 = BaggingClassifier(DecisionTreeClassifier())

sclf = StackingCVClassifier(classifiers=[c1,c2,c3],
                            meta_classifier=lr, use_probas=True)

start = time.time()
print(cross_val_score(sclf,x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

0.7691672342175991
Process takes  571.7859828472137 seconds


In [35]:
start = time.time()
c1 = RandomForestClassifier(n_estimators=100,min_samples_split=50,min_samples_leaf=1, criterion='gini', n_jobs=-1)
c2 = AdaBoostClassifier()
c3 = KNeighborsClassifier(n_neighbors=38)
c4 = KNeighborsClassifier(n_neighbors=38)
lr = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000, n_jobs=-1)

sclf = StackingCVClassifier(classifiers=[c1,c2,c3,c4],
                            meta_classifier=lr, use_probas=True)

start = time.time()
print(cross_val_score(sclf,x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

0.7753612074479902
Process takes  333.37484431266785 seconds


In [36]:
start = time.time()
c1 = RandomForestClassifier(n_estimators=100,min_samples_split=50,min_samples_leaf=1, criterion='gini', n_jobs=-1)
c2 = AdaBoostClassifier()
c3 = KNeighborsClassifier(n_neighbors=38)
c4 = DecisionTreeClassifier()
lr = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000, n_jobs=-1)

sclf = StackingCVClassifier(classifiers=[c1,c2,c3,c4],
                            meta_classifier=lr, use_probas=True)

start = time.time()
print(cross_val_score(sclf,x100,y100).mean())
print('Process takes ',time.time()-start, 'seconds')

0.7763602899282521
Process takes  352.87842202186584 seconds


In [46]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

test_data = preprocess('recipe_test.csv', convert_to_seconds=True)#pd.read_csv('COMP30027_2021_Project2_datasets\\recipe_train.csv')
test_data_100 = pd.read_csv('COMP30027_2021_Project2_datasets\\recipe_text_features_doc2vec100\\test_steps_doc2vec100.csv',header=None, names = ['coord'+str(i) for i in range(100)])
test = pd.concat([test_data,test_data_100],axis=1)
x100test = test.drop(['name','steps','ingredients'], axis=1)

imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(x100)

x100test = imp.transform(x100test)

c1 = RandomForestClassifier(n_estimators=100,min_samples_split=50,min_samples_leaf=1, criterion='gini', n_jobs=-1)
c2 = GaussianNB()#KNeighborsClassifier(n_neighbors=38)
c3 = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000, n_jobs=-1)
lr = LogisticRegression(solver='lbfgs', multi_class='multinomial', max_iter=1000, n_jobs=-1)

sclf = StackingCVClassifier(classifiers=[c1,c2,c3],
                            meta_classifier=lr, use_probas=True)

sclf.fit(x100,y100)
Y_test = sclf.predict(x100test)

out = pd.DataFrame({'duration_label':Y_test})
out.index += 1
out.to_csv('output/out_'+'randomforest_gaussiannb_logisticreg'+'.csv',index_label = 'id')

Converting...: 100%|██████████| 10000/10000 [00:11<00:00, 834.87it/s]


In [59]:
x100test

,n_steps,n_ingredients,duration_label,seconds,coord0,coord1,coord2,coord3,coord4,coord5,...,coord90,coord91,coord92,coord93,coord94,coord95,coord96,coord97,coord98,coord99
0,6,12,2.0,2400.0,-0.038488,-0.061193,0.185614,0.108549,-0.544672,-0.346892,...,-0.046046,-1.237815,0.337203,-0.478054,0.325911,0.956001,-0.016299,0.047190,-1.078757,-0.603840
1,9,5,2.0,1620.0,-0.346682,0.818242,-1.485725,-0.136444,-0.839452,-0.028747,...,-0.050925,-0.754784,-0.414631,-1.401076,0.140092,-0.593359,0.859226,0.172982,-0.032898,-0.066318
2,15,10,2.0,2100.0,-0.161517,-0.046478,-0.469720,0.856473,0.254552,0.963018,...,0.728282,-0.297418,-0.316859,-0.140749,0.103697,0.744142,-0.445749,-0.235873,0.653200,-0.108352
3,10,8,2.0,2400.0,-0.149138,-0.435168,0.398795,0.162542,0.062735,-0.489047,...,-0.898283,0.477606,-0.352423,-0.607715,0.917631,0.211922,0.168028,-0.274200,-0.593518,-0.655954
4,6,5,2.0,3600.0,-0.104086,0.423679,-0.432812,-0.721857,-0.464891,0.179705,...,0.156563,0.106496,-0.449400,-0.177187,0.419978,-0.673034,1.046548,-0.431546,-0.109882,0.228147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,6,13,1.0,5400.0,0.447364,0.079168,-0.056656,-0.241829,-0.448888,-0.198237,...,0.189909,0.501349,0.219873,0.338590,0.204403,0.633901,0.202228,0.268780,-0.752346,-0.585481
39996,15,16,2.0,4530.0,0.382064,0.291208,-0.504445,-1.015946,0.255569,0.833729,...,0.744043,0.949267,0.921160,-0.920896,-0.010590,-0.737633,-1.056501,0.675061,-0.850657,-0.424602
39997,5,8,1.0,3600.0,-0.308860,0.984259,-0.456285,-0.245439,-0.331528,-0.854780,...,0.026969,-0.494188,-0.725943,-0.377076,-0.228925,-0.092484,0.429468,-0.036762,-0.291253,0.279832
39998,7,17,2.0,4020.0,0.106077,0.089861,-0.463742,-0.865867,-0.432115,1.221770,...,-0.135346,-0.725684,0.501711,-0.143399,-0.341598,-0.770968,0.342341,0.140158,-0.511545,-0.937076
